In [1]:
import pandas as pd
import numpy as np
import json
import re

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from nltk import ngrams, everygrams

from catboost import CatBoostClassifier, Pool, cv

### Data

In [2]:
train = pd.read_json('train.json').set_index('id')

In [3]:
train

,cuisine,ingredients
id,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
22213,indian,"[water, vegetable oil, wheat, salt]"
13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."
...,...,...
29109,irish,"[light brown sugar, granulated sugar, butter, ..."
11462,italian,"[KRAFT Zesty Italian Dressing, purple onion, b..."
2238,irish,"[eggs, citrus fruit, raisins, sourdough starte..."


In [4]:
test = pd.read_json('test.json').set_index('id')

In [5]:
test

,ingredients
id,
18009,"[baking powder, eggs, all-purpose flour, raisi..."
28583,"[sugar, egg yolks, corn starch, cream of tarta..."
41580,"[sausage links, fennel bulb, fronds, olive oil..."
29752,"[meat cuts, file powder, smoked sausage, okra,..."
35687,"[ground black pepper, salt, sausage casings, l..."
...,...
30246,"[large egg yolks, fresh lemon juice, sugar, bo..."
36028,"[hot sauce, butter, sweet potatoes, adobo sauc..."
22339,"[black pepper, salt, parmigiano reggiano chees..."


In [6]:
with open('sample_submission.csv') as f:
    sample_submission = f.readlines()

In [7]:
len(sample_submission)

9945

In [8]:
sample_submission[:5]

['id,cuisine\n',
 '35203,italian\n',
 '17600,italian\n',
 '35200,italian\n',
 '17602,italian\n']

### Target: cuisine

In [9]:
len(train.cuisine.unique())

20

The sample is imbalanced:

In [10]:
train.cuisine.value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

In [11]:
train.isna().sum()

cuisine        0
ingredients    0
dtype: int64

In [12]:
train.all(axis=0)

cuisine        True
ingredients    True
dtype: bool

### Feature engineering

In [13]:
def get_ngrams(l: list, n: int):
    ngrams_list = []
    for item in l:
        grams = list(everygrams(item, max_len=n, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='<e>'))
        ngrams_list += grams
    ngrams_list1 = [''.join(i) for i in ngrams_list]
    return set(ngrams_list1)

In [14]:
def clean_some_symbols(ing_list: list):
    new_list = sum([re.sub(r'[^a-zâçèéíîú!&\'\.\®’™]', ' ', ing).split() for ing in ing_list], [])
    new_list = list(set(new_list))
    return new_list

In [15]:
def clean_letters_only(ing_list: list):
    new_list = sum([re.sub(r'[^a-zâçèéíîú]', ' ', ing).split() for ing in ing_list], [])
    new_list = list(set(new_list))
    return new_list

In [16]:
def extract_ngrams_some_symbols(old_df):
    df = old_df.copy()
    df['voc_some_symbols'] = df['ingredients'].apply(lambda x: clean_some_symbols(x))
    df['some_symb_ngrams'] = df['voc_some_symbols'].apply(lambda x: get_ngrams(x, 3))
    return df[['some_symb_ngrams']].to_dict(orient='records')

In [17]:
def extract_ngrams_letters_only(old_df):
    df = old_df.copy()
    df['voc_letters_only'] = df['ingredients'].apply(lambda x: clean_some_symbols(x))
    df['letters_only_ngrams'] = df['voc_letters_only'].apply(lambda x: get_ngrams(x, 3))
    return df[['letters_only_ngrams']].to_dict(orient='records')

In [18]:
ngrams_train = extract_ngrams_letters_only(train)
ngrams_test = extract_ngrams_letters_only(test)

In [19]:
vectorizer = DictVectorizer()

In [20]:
X = vectorizer.fit_transform(ngrams_train)
X_kaggle = vectorizer.transform(ngrams_test)

In [21]:
print(X.shape)
print(X_kaggle.shape)

(39774, 4277)
(9944, 4277)


In [22]:
X_df = pd.DataFrame(X.toarray(), index=train.index, columns=vectorizer.get_feature_names_out())
X_df.head()

,letters_only_ngrams=!,letters_only_ngrams=!<e>,letters_only_ngrams=!<e><e>,letters_only_ngrams=!®,letters_only_ngrams=!®<e>,letters_only_ngrams=&,letters_only_ngrams=&<e>,letters_only_ngrams=&<e><e>,letters_only_ngrams=&m,letters_only_ngrams=&ms,...,letters_only_ngrams=îch,letters_only_ngrams=ú,letters_only_ngrams=úa,letters_only_ngrams=úa<e>,letters_only_ngrams=’,letters_only_ngrams=’s,letters_only_ngrams=’s<e>,letters_only_ngrams=™,letters_only_ngrams=™<e>,letters_only_ngrams=™<e><e>
id,,,,,,,,,,,,,,,,,,,,,
10259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
y = train['cuisine']
y

id
10259          greek
25693    southern_us
20130       filipino
22213         indian
13162         indian
            ...     
29109          irish
11462        italian
2238           irish
41882        chinese
2362         mexican
Name: cuisine, Length: 39774, dtype: object

### CatBoost

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=6000, random_state=10)

In [25]:
train_pool = Pool(X_train, y_train)
val_pool = Pool(X_test, y_test)

In [26]:
cb = CatBoostClassifier(objective='MultiClass', eval_metric='Accuracy', task_type='GPU', random_state=10)

In [27]:
cb.fit(train_pool, eval_set=val_pool)

Learning rate set to 0.150044
0:	learn: 0.4330846	test: 0.4358333	best: 0.4358333 (0)	total: 40.2ms	remaining: 40.2s
1:	learn: 0.4595547	test: 0.4658333	best: 0.4658333 (1)	total: 80.4ms	remaining: 40.1s
2:	learn: 0.4820868	test: 0.4885000	best: 0.4885000 (2)	total: 123ms	remaining: 41s
3:	learn: 0.4897258	test: 0.4978333	best: 0.4978333 (3)	total: 167ms	remaining: 41.6s
4:	learn: 0.4952922	test: 0.5055000	best: 0.5055000 (4)	total: 206ms	remaining: 41s
5:	learn: 0.5070764	test: 0.5120000	best: 0.5120000 (5)	total: 246ms	remaining: 40.8s
6:	learn: 0.5164032	test: 0.5226667	best: 0.5226667 (6)	total: 288ms	remaining: 40.8s
7:	learn: 0.5257891	test: 0.5326667	best: 0.5326667 (7)	total: 322ms	remaining: 39.9s
8:	learn: 0.5329543	test: 0.5421667	best: 0.5421667 (8)	total: 351ms	remaining: 38.7s
9:	learn: 0.5393794	test: 0.5483333	best: 0.5483333 (9)	total: 386ms	remaining: 38.2s
10:	learn: 0.5460117	test: 0.5566667	best: 0.5566667 (10)	total: 418ms	remaining: 37.6s
11:	learn: 0.5514301	tes

94:	learn: 0.7296145	test: 0.7141667	best: 0.7141667 (94)	total: 2.83s	remaining: 27s
95:	learn: 0.7300882	test: 0.7140000	best: 0.7141667 (94)	total: 2.86s	remaining: 26.9s
96:	learn: 0.7311541	test: 0.7148333	best: 0.7148333 (96)	total: 2.88s	remaining: 26.9s
97:	learn: 0.7316871	test: 0.7141667	best: 0.7148333 (96)	total: 2.91s	remaining: 26.8s
98:	learn: 0.7325457	test: 0.7168333	best: 0.7168333 (98)	total: 2.95s	remaining: 26.8s
99:	learn: 0.7330787	test: 0.7160000	best: 0.7168333 (98)	total: 2.98s	remaining: 26.8s
100:	learn: 0.7335820	test: 0.7178333	best: 0.7178333 (100)	total: 3s	remaining: 26.7s
101:	learn: 0.7335820	test: 0.7181667	best: 0.7181667 (101)	total: 3.03s	remaining: 26.7s
102:	learn: 0.7342334	test: 0.7183333	best: 0.7183333 (102)	total: 3.06s	remaining: 26.7s
103:	learn: 0.7350329	test: 0.7188333	best: 0.7188333 (103)	total: 3.09s	remaining: 26.6s
104:	learn: 0.7357731	test: 0.7188333	best: 0.7188333 (103)	total: 3.12s	remaining: 26.6s
105:	learn: 0.7363949	test:

186:	learn: 0.7805116	test: 0.7481667	best: 0.7488333 (184)	total: 5.44s	remaining: 23.6s
187:	learn: 0.7811334	test: 0.7480000	best: 0.7488333 (184)	total: 5.47s	remaining: 23.6s
188:	learn: 0.7813999	test: 0.7485000	best: 0.7488333 (184)	total: 5.49s	remaining: 23.6s
189:	learn: 0.7813111	test: 0.7480000	best: 0.7488333 (184)	total: 5.51s	remaining: 23.5s
190:	learn: 0.7814887	test: 0.7481667	best: 0.7488333 (184)	total: 5.54s	remaining: 23.5s
191:	learn: 0.7817552	test: 0.7490000	best: 0.7490000 (191)	total: 5.56s	remaining: 23.4s
192:	learn: 0.7820217	test: 0.7488333	best: 0.7490000 (191)	total: 5.58s	remaining: 23.4s
193:	learn: 0.7826435	test: 0.7496667	best: 0.7496667 (193)	total: 5.61s	remaining: 23.3s
194:	learn: 0.7827619	test: 0.7496667	best: 0.7496667 (193)	total: 5.64s	remaining: 23.3s
195:	learn: 0.7827619	test: 0.7496667	best: 0.7496667 (193)	total: 5.67s	remaining: 23.2s
196:	learn: 0.7829691	test: 0.7493333	best: 0.7496667 (193)	total: 5.69s	remaining: 23.2s
197:	learn

279:	learn: 0.8005270	test: 0.7650000	best: 0.7650000 (279)	total: 7.85s	remaining: 20.2s
280:	learn: 0.8006455	test: 0.7646667	best: 0.7650000 (279)	total: 7.88s	remaining: 20.1s
281:	learn: 0.8008231	test: 0.7641667	best: 0.7650000 (279)	total: 7.9s	remaining: 20.1s
282:	learn: 0.8009712	test: 0.7645000	best: 0.7650000 (279)	total: 7.92s	remaining: 20.1s
283:	learn: 0.8011192	test: 0.7646667	best: 0.7650000 (279)	total: 7.94s	remaining: 20s
284:	learn: 0.8012672	test: 0.7641667	best: 0.7650000 (279)	total: 7.97s	remaining: 20s
285:	learn: 0.8012969	test: 0.7645000	best: 0.7650000 (279)	total: 7.99s	remaining: 20s
286:	learn: 0.8013265	test: 0.7643333	best: 0.7650000 (279)	total: 8.01s	remaining: 19.9s
287:	learn: 0.8017114	test: 0.7650000	best: 0.7650000 (279)	total: 8.04s	remaining: 19.9s
288:	learn: 0.8014745	test: 0.7651667	best: 0.7651667 (288)	total: 8.06s	remaining: 19.8s
289:	learn: 0.8016225	test: 0.7650000	best: 0.7651667 (288)	total: 8.08s	remaining: 19.8s
290:	learn: 0.801

372:	learn: 0.8133772	test: 0.7703333	best: 0.7706667 (338)	total: 10.1s	remaining: 17s
373:	learn: 0.8136140	test: 0.7710000	best: 0.7710000 (373)	total: 10.2s	remaining: 17s
374:	learn: 0.8138805	test: 0.7706667	best: 0.7710000 (373)	total: 10.2s	remaining: 17s
375:	learn: 0.8139397	test: 0.7708333	best: 0.7710000 (373)	total: 10.2s	remaining: 16.9s
376:	learn: 0.8141470	test: 0.7715000	best: 0.7715000 (376)	total: 10.2s	remaining: 16.9s
377:	learn: 0.8142358	test: 0.7720000	best: 0.7720000 (377)	total: 10.3s	remaining: 16.9s
378:	learn: 0.8144727	test: 0.7706667	best: 0.7720000 (377)	total: 10.3s	remaining: 16.8s
379:	learn: 0.8145023	test: 0.7711667	best: 0.7720000 (377)	total: 10.3s	remaining: 16.8s
380:	learn: 0.8145023	test: 0.7715000	best: 0.7720000 (377)	total: 10.3s	remaining: 16.8s
381:	learn: 0.8146207	test: 0.7715000	best: 0.7720000 (377)	total: 10.3s	remaining: 16.7s
382:	learn: 0.8145319	test: 0.7718333	best: 0.7720000 (377)	total: 10.4s	remaining: 16.7s
383:	learn: 0.81

...

566:	learn: 0.8300172	test: 0.7771667	best: 0.7781667 (522)	total: 14.8s	remaining: 11.3s
567:	learn: 0.8299580	test: 0.7771667	best: 0.7781667 (522)	total: 14.9s	remaining: 11.3s
568:	learn: 0.8299283	test: 0.7771667	best: 0.7781667 (522)	total: 14.9s	remaining: 11.3s
569:	learn: 0.8299283	test: 0.7775000	best: 0.7781667 (522)	total: 14.9s	remaining: 11.2s
570:	learn: 0.8298691	test: 0.7771667	best: 0.7781667 (522)	total: 14.9s	remaining: 11.2s
571:	learn: 0.8301060	test: 0.7776667	best: 0.7781667 (522)	total: 15s	remaining: 11.2s
572:	learn: 0.8300468	test: 0.7775000	best: 0.7781667 (522)	total: 15s	remaining: 11.2s
573:	learn: 0.8300468	test: 0.7775000	best: 0.7781667 (522)	total: 15s	remaining: 11.1s
574:	learn: 0.8301652	test: 0.7778333	best: 0.7781667 (522)	total: 15s	remaining: 11.1s
575:	learn: 0.8301652	test: 0.7778333	best: 0.7781667 (522)	total: 15s	remaining: 11.1s
576:	learn: 0.8301652	test: 0.7778333	best: 0.7781667 (522)	total: 15.1s	remaining: 11s
577:	learn: 0.8302244	

663:	learn: 0.8361165	test: 0.7793333	best: 0.7801667 (653)	total: 17.2s	remaining: 8.68s
664:	learn: 0.8362350	test: 0.7800000	best: 0.7801667 (653)	total: 17.2s	remaining: 8.66s
665:	learn: 0.8363830	test: 0.7796667	best: 0.7801667 (653)	total: 17.2s	remaining: 8.63s
666:	learn: 0.8365015	test: 0.7795000	best: 0.7801667 (653)	total: 17.2s	remaining: 8.61s
667:	learn: 0.8365015	test: 0.7795000	best: 0.7801667 (653)	total: 17.3s	remaining: 8.58s
668:	learn: 0.8366791	test: 0.7795000	best: 0.7801667 (653)	total: 17.3s	remaining: 8.55s
669:	learn: 0.8365903	test: 0.7795000	best: 0.7801667 (653)	total: 17.3s	remaining: 8.53s
670:	learn: 0.8366495	test: 0.7793333	best: 0.7801667 (653)	total: 17.3s	remaining: 8.5s
671:	learn: 0.8367975	test: 0.7788333	best: 0.7801667 (653)	total: 17.4s	remaining: 8.47s
672:	learn: 0.8366495	test: 0.7791667	best: 0.7801667 (653)	total: 17.4s	remaining: 8.45s
673:	learn: 0.8369752	test: 0.7791667	best: 0.7801667 (653)	total: 17.4s	remaining: 8.42s
674:	learn:

762:	learn: 0.8414757	test: 0.7810000	best: 0.7823333 (739)	total: 19.5s	remaining: 6.07s
763:	learn: 0.8415349	test: 0.7813333	best: 0.7823333 (739)	total: 19.6s	remaining: 6.04s
764:	learn: 0.8414757	test: 0.7815000	best: 0.7823333 (739)	total: 19.6s	remaining: 6.02s
765:	learn: 0.8416533	test: 0.7815000	best: 0.7823333 (739)	total: 19.6s	remaining: 5.99s
766:	learn: 0.8416533	test: 0.7813333	best: 0.7823333 (739)	total: 19.6s	remaining: 5.96s
767:	learn: 0.8418902	test: 0.7815000	best: 0.7823333 (739)	total: 19.7s	remaining: 5.94s
768:	learn: 0.8420086	test: 0.7821667	best: 0.7823333 (739)	total: 19.7s	remaining: 5.91s
769:	learn: 0.8420086	test: 0.7823333	best: 0.7823333 (739)	total: 19.7s	remaining: 5.89s
770:	learn: 0.8421271	test: 0.7823333	best: 0.7823333 (739)	total: 19.7s	remaining: 5.86s
771:	learn: 0.8422455	test: 0.7825000	best: 0.7825000 (771)	total: 19.8s	remaining: 5.84s
772:	learn: 0.8424232	test: 0.7823333	best: 0.7825000 (771)	total: 19.8s	remaining: 5.81s
773:	learn

861:	learn: 0.8470421	test: 0.7856667	best: 0.7856667 (858)	total: 21.9s	remaining: 3.5s
862:	learn: 0.8471309	test: 0.7851667	best: 0.7856667 (858)	total: 21.9s	remaining: 3.48s
863:	learn: 0.8475751	test: 0.7853333	best: 0.7856667 (858)	total: 21.9s	remaining: 3.45s
864:	learn: 0.8475751	test: 0.7853333	best: 0.7856667 (858)	total: 21.9s	remaining: 3.42s
865:	learn: 0.8476047	test: 0.7851667	best: 0.7856667 (858)	total: 22s	remaining: 3.4s
866:	learn: 0.8476343	test: 0.7853333	best: 0.7856667 (858)	total: 22s	remaining: 3.37s
867:	learn: 0.8476639	test: 0.7855000	best: 0.7856667 (858)	total: 22s	remaining: 3.35s
868:	learn: 0.8477823	test: 0.7856667	best: 0.7856667 (858)	total: 22s	remaining: 3.32s
869:	learn: 0.8477527	test: 0.7858333	best: 0.7858333 (869)	total: 22.1s	remaining: 3.29s
870:	learn: 0.8477527	test: 0.7856667	best: 0.7858333 (869)	total: 22.1s	remaining: 3.27s
871:	learn: 0.8479008	test: 0.7861667	best: 0.7861667 (871)	total: 22.1s	remaining: 3.25s
872:	learn: 0.847841

958:	learn: 0.8517499	test: 0.7868333	best: 0.7868333 (958)	total: 24.2s	remaining: 1.03s
959:	learn: 0.8520163	test: 0.7865000	best: 0.7868333 (958)	total: 24.2s	remaining: 1.01s
960:	learn: 0.8520756	test: 0.7866667	best: 0.7868333 (958)	total: 24.2s	remaining: 983ms
961:	learn: 0.8520756	test: 0.7865000	best: 0.7868333 (958)	total: 24.3s	remaining: 958ms
962:	learn: 0.8520163	test: 0.7863333	best: 0.7868333 (958)	total: 24.3s	remaining: 933ms
963:	learn: 0.8523124	test: 0.7861667	best: 0.7868333 (958)	total: 24.3s	remaining: 908ms
964:	learn: 0.8522828	test: 0.7863333	best: 0.7868333 (958)	total: 24.3s	remaining: 882ms
965:	learn: 0.8524901	test: 0.7861667	best: 0.7868333 (958)	total: 24.3s	remaining: 857ms
966:	learn: 0.8524605	test: 0.7861667	best: 0.7868333 (958)	total: 24.4s	remaining: 832ms
967:	learn: 0.8525493	test: 0.7863333	best: 0.7868333 (958)	total: 24.4s	remaining: 806ms
968:	learn: 0.8527566	test: 0.7866667	best: 0.7868333 (958)	total: 24.4s	remaining: 781ms
969:	learn

In [28]:
pred_train = cb.predict(X_train)
pred_test = cb.predict(X_test)

In [29]:
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.8554805471664594
0.7875


Overfitting - ngrams with letters + some symbols:<br>
0.8545034642032333 (train)<br>
0.7893333333333333 (val)

Overfitting - ngrams with letters only:<br>
0.8554805471664594 (train)<br>
0.7875

In [30]:
cb = CatBoostClassifier(objective='MultiClass', task_type='GPU', random_state=10)

In [31]:
cross_val_score(cb, X, y, cv=5, scoring='accuracy').mean()

Learning rate set to 0.131472
0:	learn: 2.4799668	total: 41.8ms	remaining: 41.7s
1:	learn: 2.3062171	total: 83ms	remaining: 41.4s
2:	learn: 2.1877065	total: 129ms	remaining: 42.7s
3:	learn: 2.0906148	total: 173ms	remaining: 43s
4:	learn: 2.0142038	total: 208ms	remaining: 41.4s
5:	learn: 1.9474991	total: 243ms	remaining: 40.2s
6:	learn: 1.8889981	total: 279ms	remaining: 39.6s
7:	learn: 1.8385363	total: 313ms	remaining: 38.9s
8:	learn: 1.7952702	total: 341ms	remaining: 37.6s
9:	learn: 1.7602900	total: 370ms	remaining: 36.6s
10:	learn: 1.7211650	total: 397ms	remaining: 35.7s
11:	learn: 1.6861090	total: 431ms	remaining: 35.5s
12:	learn: 1.6616585	total: 466ms	remaining: 35.4s
13:	learn: 1.6346009	total: 498ms	remaining: 35.1s
14:	learn: 1.6133064	total: 528ms	remaining: 34.6s
15:	learn: 1.5968257	total: 552ms	remaining: 33.9s
16:	learn: 1.5784574	total: 579ms	remaining: 33.5s
17:	learn: 1.5532946	total: 612ms	remaining: 33.4s
18:	learn: 1.5414929	total: 640ms	remaining: 33s
19:	learn: 1.52

162:	learn: 0.8697936	total: 4.55s	remaining: 23.4s
163:	learn: 0.8666052	total: 4.59s	remaining: 23.4s
164:	learn: 0.8649652	total: 4.62s	remaining: 23.4s
165:	learn: 0.8633046	total: 4.64s	remaining: 23.3s
166:	learn: 0.8613418	total: 4.67s	remaining: 23.3s
167:	learn: 0.8599055	total: 4.7s	remaining: 23.3s
168:	learn: 0.8591448	total: 4.72s	remaining: 23.2s
169:	learn: 0.8576759	total: 4.75s	remaining: 23.2s
170:	learn: 0.8561811	total: 4.77s	remaining: 23.1s
171:	learn: 0.8553794	total: 4.79s	remaining: 23.1s
172:	learn: 0.8541534	total: 4.82s	remaining: 23s
173:	learn: 0.8523087	total: 4.85s	remaining: 23s
174:	learn: 0.8513006	total: 4.87s	remaining: 23s
175:	learn: 0.8495905	total: 4.9s	remaining: 22.9s
176:	learn: 0.8479888	total: 4.93s	remaining: 22.9s
177:	learn: 0.8471874	total: 4.95s	remaining: 22.9s
178:	learn: 0.8460158	total: 4.98s	remaining: 22.8s
179:	learn: 0.8446451	total: 5s	remaining: 22.8s
180:	learn: 0.8438660	total: 5.02s	remaining: 22.7s
181:	learn: 0.8423586	t

324:	learn: 0.7133554	total: 8.7s	remaining: 18.1s
325:	learn: 0.7127761	total: 8.72s	remaining: 18s
326:	learn: 0.7124864	total: 8.74s	remaining: 18s
327:	learn: 0.7119376	total: 8.76s	remaining: 18s
328:	learn: 0.7111785	total: 8.79s	remaining: 17.9s
329:	learn: 0.7105028	total: 8.81s	remaining: 17.9s
330:	learn: 0.7101463	total: 8.84s	remaining: 17.9s
331:	learn: 0.7099579	total: 8.86s	remaining: 17.8s
332:	learn: 0.7093010	total: 8.88s	remaining: 17.8s
333:	learn: 0.7080299	total: 8.91s	remaining: 17.8s
334:	learn: 0.7077100	total: 8.93s	remaining: 17.7s
335:	learn: 0.7074243	total: 8.95s	remaining: 17.7s
336:	learn: 0.7069620	total: 8.98s	remaining: 17.7s
337:	learn: 0.7063373	total: 9s	remaining: 17.6s
338:	learn: 0.7059592	total: 9.02s	remaining: 17.6s
339:	learn: 0.7055470	total: 9.05s	remaining: 17.6s
340:	learn: 0.7047758	total: 9.07s	remaining: 17.5s
341:	learn: 0.7041656	total: 9.1s	remaining: 17.5s
342:	learn: 0.7040493	total: 9.12s	remaining: 17.5s
343:	learn: 0.7037987	t

491:	learn: 0.6496365	total: 12.5s	remaining: 12.9s
492:	learn: 0.6494472	total: 12.5s	remaining: 12.9s
493:	learn: 0.6494345	total: 12.5s	remaining: 12.8s
494:	learn: 0.6494221	total: 12.6s	remaining: 12.8s
495:	learn: 0.6491320	total: 12.6s	remaining: 12.8s
496:	learn: 0.6488537	total: 12.6s	remaining: 12.8s
497:	learn: 0.6482776	total: 12.6s	remaining: 12.7s
498:	learn: 0.6480393	total: 12.6s	remaining: 12.7s
499:	learn: 0.6479555	total: 12.7s	remaining: 12.7s
500:	learn: 0.6478941	total: 12.7s	remaining: 12.6s
501:	learn: 0.6478264	total: 12.7s	remaining: 12.6s
502:	learn: 0.6471434	total: 12.7s	remaining: 12.6s
503:	learn: 0.6467589	total: 12.8s	remaining: 12.6s
504:	learn: 0.6465724	total: 12.8s	remaining: 12.5s
505:	learn: 0.6462652	total: 12.8s	remaining: 12.5s
506:	learn: 0.6460427	total: 12.8s	remaining: 12.5s
507:	learn: 0.6459605	total: 12.8s	remaining: 12.4s
508:	learn: 0.6456662	total: 12.9s	remaining: 12.4s
509:	learn: 0.6455559	total: 12.9s	remaining: 12.4s
510:	learn: 

651:	learn: 0.6090109	total: 16s	remaining: 8.56s
652:	learn: 0.6087755	total: 16.1s	remaining: 8.53s
653:	learn: 0.6085016	total: 16.1s	remaining: 8.51s
654:	learn: 0.6080928	total: 16.1s	remaining: 8.48s
655:	learn: 0.6078827	total: 16.1s	remaining: 8.46s
656:	learn: 0.6078183	total: 16.2s	remaining: 8.43s
657:	learn: 0.6077438	total: 16.2s	remaining: 8.41s
658:	learn: 0.6076700	total: 16.2s	remaining: 8.38s
659:	learn: 0.6075354	total: 16.2s	remaining: 8.35s
660:	learn: 0.6073264	total: 16.2s	remaining: 8.32s
661:	learn: 0.6072326	total: 16.3s	remaining: 8.3s
662:	learn: 0.6070863	total: 16.3s	remaining: 8.27s
663:	learn: 0.6067585	total: 16.3s	remaining: 8.24s
664:	learn: 0.6065297	total: 16.3s	remaining: 8.22s
665:	learn: 0.6065175	total: 16.3s	remaining: 8.19s
666:	learn: 0.6064752	total: 16.4s	remaining: 8.16s
667:	learn: 0.6061179	total: 16.4s	remaining: 8.14s
668:	learn: 0.6057730	total: 16.4s	remaining: 8.12s
669:	learn: 0.6057078	total: 16.4s	remaining: 8.09s
670:	learn: 0.6

...

388:	learn: 0.6863013	total: 10.2s	remaining: 16s
389:	learn: 0.6861551	total: 10.2s	remaining: 15.9s
390:	learn: 0.6858912	total: 10.2s	remaining: 15.9s
391:	learn: 0.6856086	total: 10.2s	remaining: 15.9s
392:	learn: 0.6854120	total: 10.3s	remaining: 15.8s
393:	learn: 0.6847889	total: 10.3s	remaining: 15.8s
394:	learn: 0.6837142	total: 10.3s	remaining: 15.8s
395:	learn: 0.6836748	total: 10.3s	remaining: 15.8s
396:	learn: 0.6833899	total: 10.3s	remaining: 15.7s
397:	learn: 0.6829273	total: 10.4s	remaining: 15.7s
398:	learn: 0.6824194	total: 10.4s	remaining: 15.7s
399:	learn: 0.6814896	total: 10.4s	remaining: 15.6s
400:	learn: 0.6811080	total: 10.5s	remaining: 15.6s
401:	learn: 0.6804587	total: 10.5s	remaining: 15.6s
402:	learn: 0.6803057	total: 10.5s	remaining: 15.6s
403:	learn: 0.6799028	total: 10.5s	remaining: 15.5s
404:	learn: 0.6796116	total: 10.5s	remaining: 15.5s
405:	learn: 0.6787830	total: 10.6s	remaining: 15.5s
406:	learn: 0.6784838	total: 10.6s	remaining: 15.4s
407:	learn: 0.

555:	learn: 0.6282910	total: 14s	remaining: 11.2s
556:	learn: 0.6280803	total: 14s	remaining: 11.1s
557:	learn: 0.6278021	total: 14s	remaining: 11.1s
558:	learn: 0.6270438	total: 14.1s	remaining: 11.1s
559:	learn: 0.6266134	total: 14.1s	remaining: 11.1s
560:	learn: 0.6262188	total: 14.1s	remaining: 11s
561:	learn: 0.6259295	total: 14.1s	remaining: 11s
562:	learn: 0.6255861	total: 14.2s	remaining: 11s
563:	learn: 0.6252716	total: 14.2s	remaining: 11s
564:	learn: 0.6252021	total: 14.2s	remaining: 10.9s
565:	learn: 0.6248314	total: 14.2s	remaining: 10.9s
566:	learn: 0.6243077	total: 14.3s	remaining: 10.9s
567:	learn: 0.6239913	total: 14.3s	remaining: 10.9s
568:	learn: 0.6236735	total: 14.3s	remaining: 10.8s
569:	learn: 0.6233779	total: 14.3s	remaining: 10.8s
570:	learn: 0.6231446	total: 14.4s	remaining: 10.8s
571:	learn: 0.6231295	total: 14.4s	remaining: 10.8s
572:	learn: 0.6228158	total: 14.4s	remaining: 10.7s
573:	learn: 0.6223114	total: 14.4s	remaining: 10.7s
574:	learn: 0.6220544	tota

721:	learn: 0.5888629	total: 17.7s	remaining: 6.83s
722:	learn: 0.5884379	total: 17.8s	remaining: 6.8s
723:	learn: 0.5883211	total: 17.8s	remaining: 6.78s
724:	learn: 0.5882151	total: 17.8s	remaining: 6.75s
725:	learn: 0.5882051	total: 17.8s	remaining: 6.72s
726:	learn: 0.5881339	total: 17.8s	remaining: 6.7s
727:	learn: 0.5880096	total: 17.9s	remaining: 6.67s
728:	learn: 0.5878065	total: 17.9s	remaining: 6.65s
729:	learn: 0.5874371	total: 17.9s	remaining: 6.62s
730:	learn: 0.5871852	total: 17.9s	remaining: 6.6s
731:	learn: 0.5870581	total: 18s	remaining: 6.57s
732:	learn: 0.5868151	total: 18s	remaining: 6.55s
733:	learn: 0.5864357	total: 18s	remaining: 6.52s
734:	learn: 0.5862752	total: 18s	remaining: 6.5s
735:	learn: 0.5859603	total: 18s	remaining: 6.47s
736:	learn: 0.5858518	total: 18.1s	remaining: 6.45s
737:	learn: 0.5856763	total: 18.1s	remaining: 6.42s
738:	learn: 0.5855765	total: 18.1s	remaining: 6.39s
739:	learn: 0.5854832	total: 18.1s	remaining: 6.37s
740:	learn: 0.5852849	tota

888:	learn: 0.5580715	total: 21.4s	remaining: 2.68s
889:	learn: 0.5578022	total: 21.5s	remaining: 2.65s
890:	learn: 0.5576568	total: 21.5s	remaining: 2.63s
891:	learn: 0.5573773	total: 21.5s	remaining: 2.6s
892:	learn: 0.5573587	total: 21.5s	remaining: 2.58s
893:	learn: 0.5573095	total: 21.5s	remaining: 2.56s
894:	learn: 0.5572777	total: 21.6s	remaining: 2.53s
895:	learn: 0.5571358	total: 21.6s	remaining: 2.5s
896:	learn: 0.5569426	total: 21.6s	remaining: 2.48s
897:	learn: 0.5567775	total: 21.6s	remaining: 2.46s
898:	learn: 0.5567247	total: 21.7s	remaining: 2.43s
899:	learn: 0.5564138	total: 21.7s	remaining: 2.41s
900:	learn: 0.5563403	total: 21.7s	remaining: 2.38s
901:	learn: 0.5562658	total: 21.7s	remaining: 2.36s
902:	learn: 0.5559279	total: 21.7s	remaining: 2.33s
903:	learn: 0.5557405	total: 21.8s	remaining: 2.31s
904:	learn: 0.5556693	total: 21.8s	remaining: 2.29s
905:	learn: 0.5554472	total: 21.8s	remaining: 2.26s
906:	learn: 0.5551573	total: 21.8s	remaining: 2.24s
907:	learn: 0.

0.7861668990156632

Cross validation score for ngrams with letters + some symbols: 0.7858903433902703

Cross validation score for ngrams with letters only: 0.7861668990156632